# Preprocessing the data

The MERFISH mouse ileum dataset contains two important large files, molecules.csv and poly\_per\_z.json, which we first want to load into the SQL container with a spatial data type column. Unsurprisingly, the data comes essentially in a text format, and needs to be preprocessed. 

## Molecules.csv

We tried multiple approaches to this, using a python script and using SQL queries. It turned out the SQL queries were simpler and easier to execute.

In [1]:
CREATE DATABASE MouseIleum;

Commands completed successfully.

Total execution time: 00:00:00.839

In [5]:
USE MouseIleum;
DROP TABLE IF EXISTS [dbo].[Molecules];
CREATE TABLE [dbo].[Molecules](
  molecule_id int,
  gene nvarchar(8),
  x_pixel smallint,
  y_pixel smallint,
  z_pixel float,
  x_um float,
  y_um float,
  z_um float,
  area tinyint,
  total_mag float,
  brightness float, 
  qc_score float
);

BULK INSERT [dbo].[Molecules] FROM '/var/data/raw_data/molecules.csv'
WITH ( 
    FIRSTROW = 2, -- skip the column headers
    ROWS_PER_BATCH = 819665, -- however many total rows the data has
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '0x0a',
    KEEPNULLS
);

(819665 rows affected)

Total execution time: 00:00:08.165

The above code is also duplicated in importMolecules.sql.  
From here, run the processMoleculesPoints.sql script shown below.

In [2]:
USE MouseIleum;

DROP TABLE IF EXISTS MoleculesWithPoints;
SELECT molecule_id, gene, x_pixel, y_pixel, z_pixel INTO MoleculesWithPoints FROM Molecules;

ALTER TABLE MoleculesWithPoints
    ADD z_layer tinyint,
        point geometry;

UPDATE MoleculesWithPoints
    SET z_layer = CAST(ROUND(z_pixel /13.76819064, 0) AS int) + 1;

UPDATE MoleculesWithPoints
    SET point = geometry::STGeomFromText('POINT(' + CONVERT(VARCHAR(5), x_pixel) + ' ' + CONVERT(VARCHAR(5), y_pixel) + ')', 0)

(819665 rows affected)

(819665 rows affected)

(819665 rows affected)

Total execution time: 00:01:37.270

We now have a useful intermediate table MoleculesWithPoints.

# Preprocessing the polygon data

Just run the python script, get\_baysor\_polygons.py. 

Notes:

- May require some modification of the script for the file paths.
- Make sure pandas, tqdm are installed
- After running the script, move the generated csv to the processed\_data directory

In [1]:
USE MouseIleum;

DROP TABLE IF EXISTS CellPolygons;
CREATE TABLE CellPolygons (
    id int, 
    z tinyint,
    cell smallint,
    polygon_string NVARCHAR(MAX),
);

BULK INSERT CellPolygons FROM '/var/data/processed_data/baysor_SQL_polygons.csv' WITH ( FIRSTROW = 2, FIELDTERMINATOR = ',');

-- SELECT * FROM CellPolygons

(33861 rows affected)

Total execution time: 00:00:01.968

In [2]:
USE MouseIleum;
UPDATE [dbo].[CellPolygons]
    SET polygon_string = SUBSTRING(polygon_string, 2, LEN(polygon_string))
    WHERE LEFT(polygon_string,1) = '"';

(33861 rows affected)

Total execution time: 00:00:09.504

In [3]:
USE MouseIleum;
UPDATE [dbo].[CellPolygons]
    SET polygon_string = SUBSTRING(polygon_string, 1, LEN(polygon_string)-2);
    -- WHERE RIGHT(polygon_string,2) = '" ';  -- for some reason this wasn't working as expected

(33861 rows affected)

Total execution time: 00:00:00.196

The above commands are duplicated in importCellPolygons.sql.

Then, run processCellPolygons.sql (below) to get the polygons represented as geometries, not strings.

In [ ]:
USE MouseIleum;

ALTER TABLE CellPolygons
ADD polygon geometry;

GO
UPDATE CellPolygons
    SET polygon = geometry::STGeomFromText(polygon_string, 0);

ALTER TABLE CellPolygons
    DROP COLUMN polygon_string;

Now the data is in SQL, which is great.  

# Performing spatial queries

Before we can actually do spatial stuff effectively, we should add spatial indices. 

## Adding spatial indices

Before we can add spatial indices we also have to add primary keys.

In [7]:
USE MouseIleum;
ALTER TABLE MoleculesWithPoints
ADD id int identity(1,1) not null;

GO
ALTER TABLE MoleculesWithPoints
ADD CONSTRAINT molecules_id_primary_key PRIMARY KEY(id);

GO
CREATE SPATIAL INDEX PointInd ON
   [MouseIleum].[dbo].[MoleculesWithPoints](point)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = 112,YMIN = 0,XMAX = 5722, YMAX = 9393)); -- the actual min/max values in the data

Commands completed successfully.

: Msg 1913, Level 16, State 211, Line 10
The operation failed because an index or statistics with name 'PointInd' already exists on table 'MouseIleum.dbo.MoleculesWithPoints'.

Total execution time: 00:00:00.009

In [8]:
USE MouseIleum
ALTER TABLE CellPolygons
ALTER COLUMN 
    id int NOT NULL;

GO
ALTER TABLE CellPolygons
ADD CONSTRAINT polygons_id_primary_key PRIMARY KEY(id);

GO
CREATE SPATIAL INDEX PolygonInd ON
   [MouseIleum].[dbo].[CellPolygons](polygon)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = 112,YMIN = 0,XMAX = 5722, YMAX = 9393)); -- the actual min/max values in the data

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.423

## Actually running the queries

In [9]:
USE MouseIleum;
UPDATE CellPolygons
    SET polygon = polygon.MakeValid();

(33861 rows affected)

Total execution time: 00:00:11.795

In [31]:
DECLARE @startTime DATETIME
declare @endTime DATETIME
Set @starttime = getdate()

-- do the code
USE MouseIleum;
DROP TABLE IF EXISTS MoleculeCountsLayer1;
SELECT mol.gene, COUNT(mol.molecule_id) as molecule_count, poly.cell, 1 as z_layer INTO MoleculeCountsLayer1 FROM (
    SELECT * FROM [MouseIleum].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=1 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellPolygons] 
        WHERE z=1 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F
    GROUP BY mol.gene , poly.cell;

Set @endTime = GETDATE()
SELECT @EndTime - @StartTime;

GO 5

(36468 rows affected)

(1 row affected)

Total execution time: 00:00:08.325

(No column name)
1900-01-01 00:00:08.210


In [41]:
DECLARE @startTime DATETIME
declare @endTime DATETIME
Set @starttime = getdate()

-- do the code
USE MouseIleum;
DROP TABLE IF EXISTS MoleculeCountsAllLayers;
SELECT mol.molecule_id, mol.gene, mol.point, poly.cell as cell_id, poly.polygon as cell_polygon, 1 as z_layer INTO MoleculeCountsAllLayers FROM (
    SELECT * FROM [MouseIleum].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=1 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellPolygons] 
        WHERE z=1 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F

Set @endTime = GETDATE()
SELECT @EndTime - @StartTime;

GO 1

(93131 rows affected)

(1 row affected)

Total execution time: 00:00:07.698

(No column name)
1900-01-01 00:00:07.687


In [40]:
USE MouseIleum;


SELECT * INTO #Whatever FROM (
    SELECT * FROM [MouseIleum].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=2 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellPolygons] 
        WHERE z=2 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F

-- INSERT INTO MoleculeCountsAllLayers (molecule_id, gene, point, cell_id, cell_polygon, z_layer)
-- SELECT mol.molecule_id, mol.gene, mol.point, poly.cell as cell_id, poly.polygon as cell_polygon, 2 as z_layer INTO MoleculeCountsAllLayers FROM #Whatever


: Msg 2705, Level 16, State 3, Line 4
Column names in each table must be unique. Column name 'id' in table '#Whatever' is specified more than once.

Total execution time: 00:00:00.027

# Convex Hulls

and polygon validity

In [21]:
USE MouseIleum;

DECLARE @point1 geometry;
set @point1 = (SELECT TOP(1) point from MoleculesWithPoints where molecule_id = 1);

SELECT @point1.STAsText();
SELECT * from MoleculesWithPoints where molecule_id = 1;

SELECT * FROM CellPolygons
where polygon.STIntersects(@point1) = 1;



(1 row affected)

(1 row affected)

(7 rows affected)

Total execution time: 00:00:00.075

(No column name)
POINT (1705 1271)


molecule_id,gene,x_pixel,y_pixel,z_pixel,z_layer,point,id
1,Maoa,1705,1271,0,1,0x00000000010C0000000000A49A400000000000DC9340,38976


id,z,cell,polygon
25362,7,28,0x000000000104510000000000000000049B4000000000004094400000000000109B40000000000040944000000000001C9B4000000000004094400000000000289B4000000000004094400000000000349B4000000000004094400000000000409B40000000000040944000000000004C9B4000000000004094400000000000589B4000000000004094400000000000649B4000000000004094400000000000649B4000000000003494400000000000709B4000000000002894400000000000709B4000000000001C94400000000000709B40000000000010944000000000007C9B40000000000004944000000000007C9B400000000000F8934000000000007C9B400000000000EC934000000000007C9B400000000000E093400000000000709B400000000000D493400000000000709B400000000000C893400000000000649B400000000000BC93400000000000649B400000000000B093400000000000589B400000000000A493400000000000589B40000000000098934000000000004C9B4000000000009893400000000000409B4000000000009893400000000000349B4000000000009893400000000000289B40000000000098934000000000001C9B4000000000009893400000000000109B4000000000009893400000000000049B4000000000009893400000000000F89A4000000000009893400000000000EC9A4000000000008C93400000000000E09A4000000000008C93400000000000D49A4000000000008C93400000000000C89A4000000000008C93400000000000BC9A4000000000008093400000000000B09A4000000000008093400000000000A49A4000000000008093400000000000989A40000000000080934000000000008C9A4000000000007493400000000000809A4000000000007493400000000000749A4000000000007493400000000000689A40000000000068934000000000005C9A4000000000006893400000000000509A4000000000006893400000000000449A4000000000005C93400000000000389A4000000000005C934000000000002C9A4000000000006893400000000000209A4000000000007493400000000000209A4000000000008093400000000000149A4000000000008C93400000000000149A4000000000009893400000000000149A400000000000A493400000000000149A400000000000B093400000000000149A400000000000BC93400000000000149A400000000000C893400000000000149A400000000000D493400000000000149A400000000000E093400000000000149A400000000000EC93400000000000149A400000000000F893400000000000209A40000000000004944000000000002C9A4000000000001094400000000000389A4000000000001C94400000000000449A4000000000002894400000000000449A4000000000003494400000000000509A40000000000034944000000000005C9A4000000000003494400000000000689A4000000000003494400000000000749A4000000000003494400000000000809A40000000000034944000000000008C9A4000000000004094400000000000989A4000000000004094400000000000A49A4000000000004094400000000000B09A4000000000004094400000000000BC9A4000000000003494400000000000C89A4000000000003494400000000000D49A4000000000003494400000000000E09A4000000000003494400000000000EC9A4000000000003494400000000000F89A4000000000003494400000000000049B40000000000040944001000000020000000001000000FFFFFFFF0000000003
3942,2,77,0x000000000104520000000000000000EC9A4000000000003494400000000000F89A4000000000003494400000000000049B4000000000003494400000000000109B40000000000034944000000000001C9B4000000000003494400000000000289B4000000000003494400000000000349B4000000000003494400000000000349B4000000000002894400000000000349B4000000000001C94400000000000349B4000000000001094400000000000349B4000000000000494400000000000349B400000000000F893400000000000349B400000000000EC93400000000000409B400000000000E093400000000000409B400000000000D493400000000000409B400000000000C893400000000000409B400000000000BC93400000000000409B400000000000B0934000000000004C9B400000000000A4934000000000004C9B40000000000098934000000000004C9B4000000000008C934000000000004C9B40000000000080934000000000004C9B40000000000074934000000000004C9B40000000000068934000000000004C9B4000000000005C934000000000004C9B4000000000005093400000000000409B4000000000004493400000000000349B4000000000003893400000000000289B40000000000038934000000000001C9B4000000000002C93400000000000109B4000000000002093400000000000049B4000000000001493400000000000049B4000000000000893400000000000F89A4000000000000893400000000000EC9A4000000000000893400000000000E09A400000000000FC92400000000000D49A400000000000FC92400000000000C89A400000000000F092400000000000BC9A400000000000E492400000000000B09A400000000000D89240